In [6]:
import numpy as np
import pandas as pd

In [7]:
df=pd.read_csv('updated_version2.csv')

In [8]:

df.shape

(54068, 19)

## Text preprocessing Pipeline Using SpaCy

In [9]:
import pandas as pd
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from transformers import AutoTokenizer
import re
import emoji

# Load spaCy (no parser/ner for speed)

nlp = spacy.load("en_core_web_sm", disable=["ner", "parser", "textcat"])

# Load BERTweet tokenizer
bertweet_tok = AutoTokenizer.from_pretrained("vinai/bertweet-base", use_fast=False)

# Keep negations
negations = {"no", "not", "nor", "n't"}
context_words = {
    # First-person
    "i", "me", "my", "mine", "myself", "we", "us", "our", "ours", "ourselves",

    # Second-person
    "you", "your", "yours", "yourself", "yourselves",

    # Third-person singular
    "he", "him", "his", "himself",
    "she", "her", "hers", "herself",
    "it", "its", "itself",

    # Third-person plural
    "they", "them", "their", "theirs", "themselves",

    # Demonstratives
    "this", "that", "these", "those",

    # Interrogatives / relatives
    "who", "whom", "whose", "which", "what",

    # Possessive determiners often pruned but useful in context
    "own",

    # Common discourse/contextual markers
    "here", "there", "where", "when", "why", "how"
}

stop_words = STOP_WORDS - negations-context_words

# Emoji regex (keeps most Unicode emojis)
emoji_pattern = re.compile("["
    u"\U0001F600-\U0001F64F"  # emoticons
    u"\U0001F300-\U0001F5FF"  # symbols & pictographs
    u"\U0001F680-\U0001F6FF"  # transport & map
    u"\U0001F1E0-\U0001F1FF"  # flags
    u"\U00002700-\U000027BF"  # dingbats
    u"\U0001F900-\U0001F9FF"  # supplemental symbols
    u"\U00002600-\U000026FF"  # misc symbols
    "]+", flags=re.UNICODE)

def emoji_to_text(e: str) -> str:
    """
    Convert emoji to its text alias using emoji.demojize
    e.g., 😂 -> ":face_with_tears_of_joy:"
    """
    return emoji.demojize(e)
def is_emoji(token: str) -> bool:
    return bool(emoji_pattern.fullmatch(token))

def normalize_punct(tok: str) -> str | None:
    """Normalize emotional punctuation runs."""
    if set(tok) == {"!"}:
        return "!"
    if set(tok) == {"?"}:
        return "?"
    if set(tok) == {"."} and len(tok) >= 3:
        return "..."
    return None

def in_bertweet(token: str) -> bool:
    """Check if token is in BERTweet vocab (not [UNK])."""
    return bertweet_tok.convert_tokens_to_ids(token) != bertweet_tok.unk_token_id

def preprocessing(doc):
    tokens = []

    last_punct = None      # collapse !!!!! or ??? or ...
    last_emoji = None      # collapse repeated emojis

    for tok in doc:
        t = tok.text
        lower = t.lower()

        # ---------------------------
        # EMOJIS (keep one per run)
        # ---------------------------
        if is_emoji(t):
            alias = emoji_to_text(t)   # 😂 -> face_with_tears_of_joy
            if alias != last_emoji and in_bertweet(alias):
                tokens.append(alias)
                last_emoji = alias
            last_punct = None
            continue

        # ---------------------------
        # PUNCTUATION (keep ! ? ...)
        # ---------------------------
        if tok.is_punct:
            if t in {"!", "?", "..."}:
                if t != last_punct:
                    tokens.append(t)
                    last_punct = t
            else:
                last_punct = None
            last_emoji = None
            continue

        # ---------------------------
        # NEGATION handling
        # didn't -> not
        # ---------------------------
        if lower in {"n't", "'nt"}:
            tokens.append("not")
            last_punct = None
            last_emoji = None
            continue

        # ---------------------------
        # NUMBERS (keep)
        # useful for health/metrics
        # ---------------------------
        if tok.like_num:
            tokens.append(lower)
            last_punct = None
            last_emoji = None
            continue

        # ---------------------------
        # WORDS
        # ---------------------------
        if tok.is_alpha:
            lemma = tok.lemma_.lower()

            # fix spaCy pronoun bug
            if lemma == "-pron-":
                lemma = lower

            if lemma not in stop_words and in_bertweet(lemma):
                tokens.append(lemma)

            last_punct = None
            last_emoji = None
            continue

        # reset runs for anything else
        last_punct = None
        last_emoji = None

    return tokens


# --- Apply to DataFrame ---
text_col = "cleaned_text"
wellness_cols = df.drop(columns=['text']).columns

corpus = []
for doc in nlp.pipe(df['text'].astype(str), batch_size=512):
    tokens = preprocessing(doc)
    corpus.append(" ".join(tokens))

processed_df = pd.DataFrame({"cleaned_text": corpus}, index=df.index)
processed_df = pd.concat([processed_df, df[wellness_cols]], axis=1)



In [ ]:
'''
import re
import pandas as pd
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from transformers import AutoTokenizer
import emoji

# Load spaCy (no parser/ner for speed)
nlp = spacy.load("en_core_web_sm", disable=["ner", "parser", "textcat"])

# Load BERTweet tokenizer
bertweet_tok = AutoTokenizer.from_pretrained("vinai/bertweet-base", use_fast=False)

# Emoji regex (keeps most Unicode emojis)
emoji_pattern = re.compile("["
    u"\U0001F600-\U0001F64F"  # emoticons
    u"\U0001F300-\U0001F5FF"  # symbols & pictographs
    u"\U0001F680-\U0001F6FF"  # transport & map
    u"\U0001F1E0-\U0001F1FF"  # flags
    u"\U00002700-\U000027BF"  # dingbats
    u"\U0001F900-\U0001F9FF"  # supplemental symbols
    u"\U00002600-\U000026FF"  # misc symbols
    "]+", flags=re.UNICODE)

def emoji_to_text(e: str) -> str:
    """
    Convert emoji to its text alias using emoji.demojize
    e.g., 😂 -> ":face_with_tears_of_joy:"
    """
    return emoji.demojize(e)
def is_emoji(token: str) -> bool:
    return bool(emoji_pattern.fullmatch(token))

def normalize_punct(tok: str) -> str | None:
    """Normalize emotional punctuation runs."""
    if set(tok) == {"!"}:
        return "!"
    if set(tok) == {"?"}:
        return "?"
    if set(tok) == {"."} and len(tok) >= 3:
        return "..."
    return None

def in_bertweet(token: str) -> bool:
    """Check if token is in BERTweet vocab (not [UNK])."""
    return bertweet_tok.convert_tokens_to_ids(token) != bertweet_tok.unk_token_id

def preprocessing(doc):
    tokens = []
    last_punct = None    # track last emitted '!' or '?'
    last_emoji = None    # track last emitted emoji alias

    for tok in doc:
        t = tok.text

        # --- EMOJIS (one emoji per token; collapse consecutive repeats) ---
        if is_emoji(t):
            alias = emoji_to_text(t)              # e.g. 😂 -> 'face_with_tears_of_joy'
            if in_bertweet(alias) and alias != last_emoji:
                tokens.append(alias)
                last_emoji = alias
            # break punctuation run regardless
            last_punct = None
            continue

        # --- PUNCT (keep only ! and ?; collapse consecutive repeats) ---
        if tok.is_punct:
            if t in {"!", "?","..."}:
                if t != last_punct:
                    tokens.append(t)             # keep the first in a run
                    last_punct = t
            else:
                # ignore other punctuation; also break the run
                last_punct = None
            # break emoji run
            last_emoji = None
            continue

        # --- CONTRACTION NEGATION (spaCy splits "didn't" -> "did" + "n't") ---
        if t.lower() in {"n't", "'nt"}:
            tokens.append("not")
            last_punct = None
            last_emoji = None
            continue

        # --- WORDS ---
        if tok.is_alpha:
            lemma = tok.lemma_.lower()
            if lemma == "-pron-":
                lemma = t.lower()
            if lemma not in stop_words and in_bertweet(lemma):
                tokens.append(lemma)
            last_punct = None
            last_emoji = None
            continue

        # anything else breaks runs
        last_punct = None
        last_emoji = None

    return tokens



# --- Apply to DataFrame ---
text_col = "cleaned_text"
emotion_cols = processed_df.drop(columns=['text']).columns

corpus = []
for doc in nlp.pipe(processed_df['text'].astype(str), batch_size=512):
    tokens = preprocessing(doc)
    corpus.append(" ".join(tokens))

processed_df = pd.DataFrame({"cleaned_text": corpus}, index=processed_df.index)
processed_df = pd.concat([processed_df, processed_df[emotion_cols]], axis=1)

'''

KeyError: "['text'] not found in axis"

In [10]:
processed_df.head()

,cleaned_text,physical_wellness,not_physical_wellness,intellectual_wellness,occupational_wellness,not_occupational_wellness,financial_wellness,not_financial_wellness,social_interaction_wellness,not_social_interaction_wellness,spiritual_wellness,mental_wellness,not_mental_wellness,compassion_contribution_wellness,family_and_caregiving,not_family_and_caregiving,leisure_and_travel,life_events_transitions,neutral
0,teacher i stumble home exhausted numb like day...,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,my manager tiny task i nurse it i feel like ro...,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
2,project software engineer feel like rerun ther...,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
3,i this cashier job paycheck little work not me...,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0
4,culture my warehouse team toxic comment gossip...,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0


In [11]:
if 'Unnamed: 0' in processed_df.columns:
    processed_df.drop('Unnamed: 0', axis=1, inplace=True)

In [13]:
processed_df.columns

Index(['cleaned_text', 'physical_wellness', 'not_physical_wellness',
       'intellectual_wellness', 'occupational_wellness',
       'not_occupational_wellness', 'financial_wellness',
       'not_financial_wellness', 'social_interaction_wellness',
       'not_social_interaction_wellness', 'spiritual_wellness',
       'mental_wellness', 'not_mental_wellness',
       'compassion_contribution_wellness', 'family_and_caregiving',
       'not_family_and_caregiving', 'leisure_and_travel',
       'life_events_transitions', 'neutral'],
      dtype='object')

In [14]:
y=processed_df.drop('cleaned_text',axis=1)


In [15]:
y.head()

,physical_wellness,not_physical_wellness,intellectual_wellness,occupational_wellness,not_occupational_wellness,financial_wellness,not_financial_wellness,social_interaction_wellness,not_social_interaction_wellness,spiritual_wellness,mental_wellness,not_mental_wellness,compassion_contribution_wellness,family_and_caregiving,not_family_and_caregiving,leisure_and_travel,life_events_transitions,neutral
0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0


In [16]:
import pandas as pd
import numpy as np
import torch, re
from transformers import AutoTokenizer, AutoModel
from torch.utils.data import Dataset, DataLoader

MODEL_NAME = "vinai/bertweet-base"   # 768-d; use "vinai/bertweet-large" for 1024-d
DEVICE = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"


label_cols = processed_df.drop('cleaned_text', axis=1).columns.tolist()

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)
model = AutoModel.from_pretrained(MODEL_NAME).to(DEVICE)
model.eval()

USER_RE = re.compile(r"@\w+")
URL_RE  = re.compile(r"http\S+|www\.\S+")
def normalize_tweet(t: str) -> str:
    t = USER_RE.sub("<user>", t)
    t = URL_RE.sub("<url>", t)
    return t

## Model Building and Making it ready for training
1. Selective layer freezing/unfreezing (top 3 unfrozen)
2. Strided tokenization for long sequences (>128)

3. Custom Dataset and batch collate

4. Dynamic padding and truncation

5. Class balancing with pos_weight

6. Mean-pooling sentence representation

7. Lightweight dropout + linear classification head



In [17]:
!pip install peft


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.0/557.0 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [peft]1/2 [peft]


## Working Training part using 3 unfreezed layers

In [18]:
# Dataset: return raw text + labels only (no tokenization here)
from torch.utils.data import random_split
import re, copy, math, torch, torch.nn as nn, torch.optim as optim
from transformers import AutoModel, AutoTokenizer
from peft import LoraConfig, get_peft_model, TaskType
import re
class LabeledTweetDataset(Dataset):
    def __init__(self, df, text_col: str, label_cols, max_len: int = 128):
        self.texts  = df[text_col].astype(str).tolist()
        self.labels = df[label_cols].astype("float32").values
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, i):
        return {
            "text": normalize_tweet(self.texts[i]),
            "labels": torch.from_numpy(self.labels[i])
        }

# Collate: tokenize the whole batch at once (fast) and pad
def build_collate(tokenizer, max_len=128, stride=None):
    def collate(batch):
        texts  = [b["text"] for b in batch]
        labels = torch.stack([b["labels"] for b in batch], dim=0)

        kwargs = dict(
            truncation=True,
            max_length=max_len,
            padding=True,           # dynamic padding to longest in this batch
            return_attention_mask=True,
            return_tensors="pt"
        )
        if stride is not None:
            kwargs.update(
                stride=stride,
                return_overflowing_tokens=True
            )
        enc = tokenizer(texts, padding=True,truncation=True,max_length=128,return_tensors="pt")

        # If using stride, duplicate labels per overflow chunk
        if "overflow_to_sample_mapping" in enc:
            mapping = enc["overflow_to_sample_mapping"]
            labels = labels[mapping]

        enc["labels"] = labels
        return enc
    return collate
max_len   = 128
batch_sz  = 16
dataset   = LabeledTweetDataset(processed_df, text_col=text_col, label_cols=label_cols, max_len=max_len)

val_frac  = 0.2
val_size  = int(len(dataset) * val_frac)
train_size = len(dataset) - val_size

train_ds, val_ds = random_split(dataset, [train_size, val_size])


collate_fn = build_collate(tokenizer, max_len=128, stride=32)  # or stride=None
train_loader = DataLoader(train_ds, batch_size=16, shuffle=True,collate_fn=collate_fn)
val_loader   = DataLoader(val_ds,   batch_size=16, shuffle=False, num_workers=4,collate_fn=collate_fn)


# ==== 0) Imports ====
##------------------------TRAINING STARTS HERE##-------------------


# ==== 1) Class weights (your code, with dtype/device)
with torch.no_grad():
    Y_np = processed_df[label_cols].astype("float32").values
Y_t = torch.tensor(Y_np, dtype=torch.float32)
N   = Y_t.shape[0]
pos_counts = Y_t.sum(dim=0)
neg_counts = N - pos_counts
pos_weight = (neg_counts / (pos_counts + 1e-8)).to(DEVICE).float()

class BertweetClassifier(nn.Module):
    def __init__(self, model_name="vinai/bertweet-base", num_labels=18,
                 lora_r=8, lora_alpha=16, lora_dropout=0.05):
        super().__init__()
        # Base encoder
        self.encoder = AutoModel.from_pretrained(model_name)  # Roberta-like (BERTweet)

        for n, _ in self.encoder.named_modules():
          if "layer.0" in n:   # show where the first layer lives
              print(n)
              break


        # Freeze base weights in ALL layers first; PEFT adapters remain trainable
        for n, p in self.encoder.named_parameters():
                p.requires_grad = False
        total_layers = self.encoder.config.num_hidden_layers
        for idx in range(total_layers - 3, total_layers):   # 9,10,11 for 12-layer base
            for p in self.encoder.encoder.layer[idx].parameters():
                p.requires_grad = True

        # Classification head
        hidden_size = self.encoder.config.hidden_size  # 768 for bertweet-base
        self.dropout = nn.Dropout(0.1)
        self.classifier = nn.Linear(hidden_size, num_labels)

    def forward(self, input_ids, attention_mask):
        out = self.encoder(input_ids=input_ids, attention_mask=attention_mask)
        # mean-pool over valid tokens
        mask = attention_mask.unsqueeze(-1)
        x = (out.last_hidden_state * mask).sum(1) / mask.sum(1).clamp(min=1e-6)
        x = self.dropout(x)
        return self.classifier(x)  # logits [B, num_labels]

model = BertweetClassifier(num_labels=len(label_cols)).to(DEVICE)


encoder.layer.0


## testing with peft

In [19]:
'''import torch
import torch.nn as nn
from transformers import AutoModel
from peft import LoraConfig, get_peft_model, TaskType
# Dataset: return raw text + labels only (no tokenization here)
from torch.utils.data import random_split
from transformers import AutoModel, AutoTokenizer

class LabeledTweetDataset(Dataset):
    def __init__(self, df, text_col: str, label_cols, max_len: int = 128):
        self.texts  = df[text_col].astype(str).tolist()
        self.labels = df[label_cols].astype("float32").values
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, i):
        return {
            "text": normalize_tweet(self.texts[i]),
            "labels": torch.from_numpy(self.labels[i])
        }

# Collate: tokenize the whole batch at once (fast) and pad
def build_collate(tokenizer, max_len=128, stride=None):
    def collate(batch):
        texts  = [b["text"] for b in batch]
        labels = torch.stack([b["labels"] for b in batch], dim=0)

        kwargs = dict(
            truncation=True,
            max_length=max_len,
            padding=True,           # dynamic padding to longest in this batch
            return_attention_mask=True,
            return_tensors="pt"
        )
        if stride is not None:
            kwargs.update(
                stride=stride,
                return_overflowing_tokens=True
            )
        enc = tokenizer(texts, padding=True,truncation=True,max_length=128,return_tensors="pt")

        # If using stride, duplicate labels per overflow chunk
        if "overflow_to_sample_mapping" in enc:
            mapping = enc["overflow_to_sample_mapping"]
            labels = labels[mapping]

        enc["labels"] = labels
        return enc
    return collate
max_len   = 128
batch_sz  = 16
dataset   = LabeledTweetDataset(processed_df, text_col=text_col, label_cols=label_cols, max_len=max_len)

val_frac  = 0.2
val_size  = int(len(dataset) * val_frac)
train_size = len(dataset) - val_size

train_ds, val_ds = random_split(dataset, [train_size, val_size])


collate_fn = build_collate(tokenizer, max_len=128, stride=32)  # or stride=None
train_loader = DataLoader(train_ds, batch_size=16, shuffle=True,collate_fn=collate_fn)
val_loader   = DataLoader(val_ds,   batch_size=16, shuffle=False,collate_fn=collate_fn)


# ==== 0) Imports ====
##------------------------TRAINING STARTS HERE##-------------------

# ==== 1) Class weights (your code, with dtype/device)
with torch.no_grad():
    Y_np = processed_df[label_cols].astype("float32").values
Y_t = torch.tensor(Y_np, dtype=torch.float32)
N   = Y_t.shape[0]
pos_counts = Y_t.sum(dim=0)
neg_counts = N - pos_counts
pos_weight = (neg_counts / (pos_counts + 1e-8)).to(DEVICE).float()

class BertweetClassifier(nn.Module):
    def __init__(self, model_name="vinai/bertweet-base", num_labels=18,
                 lora_r=8, lora_alpha=16, lora_dropout=0.05):
        super().__init__()

        # 1) Load the base encoder
        base_encoder = AutoModel.from_pretrained(model_name)  # BERTweet = RoBERTa architecture

        # 2) Freeze ALL base weights
        for p in base_encoder.parameters():
            p.requires_grad = False

        # 3) Attach LoRA adapters (these will be the ONLY trainable params inside the encoder)
        # For RoBERTa/BERTweet, typical attention linear layers are named: query, key, value, and output.dense
        # You can safely target: ["query", "key", "value", "dense"] (matches attention projections + output)
        lora_cfg = LoraConfig(
            r=lora_r,
            lora_alpha=lora_alpha,
            lora_dropout=lora_dropout,
            bias="none",
            task_type=TaskType.SEQ_CLS,
        )
        ''''''     config = LoraConfig(
          r=16, #attention heads
          lora_alpha=32, #alpha scaling
          # target_modules=["q_proj", "v_proj"], #if you know the
          lora_dropout=0.05,
          bias="none",
          task_type="CAUSAL_LM" # set this for CLM or Seq2Seq
          )''''''
        self.encoder = get_peft_model(base_encoder, lora_cfg)

        # 4) Classification head (kept trainable)
        hidden_size = self.encoder.config.hidden_size  # 768 for bertweet-base
        self.dropout = nn.Dropout(0.1)
        self.classifier = nn.Linear(hidden_size, num_labels)

        # (optional) sanity check: print trainable parameter count
        trainable, total = 0, 0
        for n, p in self.named_parameters():
            total += p.numel()
            if p.requires_grad:
                trainable += p.numel()
        print(f"Trainable params: {trainable:,} / {total:,}")


    def forward(self, input_ids, attention_mask):
        out = self.encoder(input_ids=input_ids, attention_mask=attention_mask)
        # mean-pool over valid tokens
        mask = attention_mask.unsqueeze(-1)
        x = (out.last_hidden_state * mask).sum(1) / mask.sum(1).clamp(min=1e-6)
        x = self.dropout(x)
        return self.classifier(x)  # logits [B, num_labels]


model = BertweetClassifier(num_labels=len(label_cols)).to(DEVICE)
'''

'import torch\nimport torch.nn as nn\nfrom transformers import AutoModel\nfrom peft import LoraConfig, get_peft_model, TaskType\n# Dataset: return raw text + labels only (no tokenization here)\nfrom torch.utils.data import random_split\nfrom transformers import AutoModel, AutoTokenizer\n\nclass LabeledTweetDataset(Dataset):\n    def __init__(self, df, text_col: str, label_cols, max_len: int = 128):\n        self.texts  = df[text_col].astype(str).tolist()\n        self.labels = df[label_cols].astype("float32").values\n        self.max_len = max_len\n\n    def __len__(self):\n        return len(self.texts)\n\n    def __getitem__(self, i):\n        return {\n            "text": normalize_tweet(self.texts[i]),\n            "labels": torch.from_numpy(self.labels[i])\n        }\n\n# Collate: tokenize the whole batch at once (fast) and pad\ndef build_collate(tokenizer, max_len=128, stride=None):\n    def collate(batch):\n        texts  = [b["text"] for b in batch]\n        labels = torch.stac

In [20]:
import re

def print_trainable_breakdown(model):
    # robust layer matcher for BERT/RoBERTa/BERTweet style names (works with PEFT wrappers too)
    layer_rx = re.compile(
        r"(?:^|\.)(?:encoder\.layer|roberta\.encoder\.layer|bert\.encoder\.layer|model\.encoder\.layers)\.(\d+)\."
    )

    total = 0
    trainable_total = 0
    peft_trainable = 0                  # LoRA-only (lora_A/lora_B/etc.)
    top3_base_trainable = 0             # non-LoRA params in layers 9..11

    for name, p in model.named_parameters():
        n_params = p.numel()
        total += n_params
        if p.requires_grad:
            trainable_total += n_params
            if "lora_" in name:
                peft_trainable += n_params
            else:
                m = layer_rx.search(name)
                if m and int(m.group(1)) >= 9:   # last three layers
                    top3_base_trainable += n_params

    pct = 100.0 * trainable_total / total if total else 0.0
    fmt = lambda x: f"{x:,}"

    print(f"Total params:                  {fmt(total)}")
    print(f"Trainable params:              {fmt(trainable_total)} ({pct:.2f}%)")
    print(f"  ├─ PEFT (LoRA) trainable:    {fmt(peft_trainable)}")
    print(f"  └─ Top-3 base trainable:     {fmt(top3_base_trainable)}  (layers 9–11, non-LoRA)")

# usage
print_trainable_breakdown(model)


Total params:                  134,913,810
Trainable params:              21,277,458 (15.77%)
  ├─ PEFT (LoRA) trainable:    0
  └─ Top-3 base trainable:     21,263,616  (layers 9–11, non-LoRA)


In [21]:
from torch.optim.lr_scheduler import StepLR
import torch.optim as optim


opt = optim.AdamW(model.parameters(), lr=2e-5, weight_decay=1e-4)
scheduler = StepLR(opt, step_size=5, gamma=0.1)  # every 5 epochs, LR *= 0.1
criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
best_val = float("inf")   # initialize as +∞ (so any first val_loss is smaller)
best_state = None         # to store the best model weights
patience = 5              # how many epochs without improvement before stopping
bad = 0                   #

epochs = 25
for epoch in range(epochs):
    model.train()
    running_loss = 0.0

    for batch in train_loader:
        yb = batch["labels"].to(DEVICE)                 # don't mutate `batch` with pop
        inputs = {
            "input_ids": batch["input_ids"].to(DEVICE),
            "attention_mask": batch["attention_mask"].to(DEVICE),
        }
        # guardrails:
        assert "labels" not in inputs

        opt.zero_grad(set_to_none=True)
        logits = model(**inputs)                         # only id/mask can enter
        loss   = criterion(logits, yb)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        opt.step()

        running_loss += loss.item()


    train_loss = running_loss / len(train_loader)

    # --- Validate (no grads) ---
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for batch in val_loader:
            yb = batch["labels"].to(DEVICE)
            inputs = {
                "input_ids": batch["input_ids"].to(DEVICE),
                "attention_mask": batch["attention_mask"].to(DEVICE),
            }
            logits = model(**inputs)
            val_loss += criterion(logits, yb).item()
    val_loss /= len(val_loader)


    # Use val loss to drive LR and early stopping
    scheduler.step(val_loss)
    #best_val = val_loss
    #best_state = {k: v.detach().cpu().clone() for k, v in model.state_dict().items()}


    if val_loss < best_val - 1e-6:
            best_val = val_loss
            best_state = {k: v.detach().cpu().clone() for k, v in model.state_dict().items()}
            bad = 0
    else:
        bad += 1
        if bad >= patience:
            print("Early stopping. Restoring best checkpoint.")
            model.load_state_dict(best_state)
            break

    print(f"Epoch {epoch+1}/{epochs} - train: {train_loss:.4f} - val: {val_loss:.4f} - lr: {opt.param_groups[0]['lr']:.2e}")


AttributeError: Can't pickle local object 'build_collate.<locals>.collate'

In [ ]:
import torch

# === Helper for prediction with probabilities ===
def predict(texts, model, tokenizer, label_cols, max_len=128, threshold=0.5, device="cuda"):
    model.eval()
    with torch.no_grad():
        enc = tokenizer(
            texts,
            truncation=True,
            max_length=max_len,
            padding=True,
            return_tensors="pt"
        ).to(device)

        logits = model(enc["input_ids"], enc["attention_mask"])
        probs = torch.sigmoid(logits)  # [B, num_labels]
        preds = (probs > threshold).int()

    results = []
    for text, row_probs, row_preds in zip(texts, probs, preds):
        prob_dict = {label_cols[i]: float(row_probs[i]) for i in range(len(label_cols))}
        pred_labels = [label_cols[i] for i, v in enumerate(row_preds.tolist()) if v == 1]
        results.append({
            "text": text,
            "labels": pred_labels,
            "probabilities": prob_dict
        })
    return results


# === Example usage ===
sample_texts = ["he is not well"]

'''My brother has been eating more fruits and seems healthier lately.

The stranger I met at the park said running helps him clear his mind.

She finally took a vacation, and it did wonders for her stress levels.

Our neighbor spends every evening walking his dog and looks more active now."]'''

predictions = predict(sample_texts, model, tokenizer, label_cols, max_len=128, device=DEVICE)

for pred in predictions:
    print(f"Text: {pred['text']}")
    print(f"Predicted labels: {pred['labels']}")
    print("Probabilities:")
    for lbl, score in pred["probabilities"].items():
      if score>0.5:
        print(f"  {lbl}: {score:.3f}")
    print()


In [ ]:
import os, json, torch
from pathlib import Path

export_dir = Path("bertweet_multilabel_export")
export_dir.mkdir(parents=True, exist_ok=True)

# 1) weights
torch.save(model.state_dict(), export_dir / "pytorch_model.bin")

# 2) tokenizer (recreate the same tokenizer object name you used)
# If you already have `tokenizer`, just:
tokenizer.save_pretrained(export_dir)

# 3) metadata you’ll need at inference time
meta = {
    "model_name": "vinai/bertweet-base",
    "num_labels": len(label_cols),
    "label_cols": list(label_cols),
    # replace with your tuned thresholds if you have them; else 0.5 default
    "thresholds": [0.5] * len(label_cols),
    "max_len": 128
}
with open(export_dir / "config.json", "w") as f:
    json.dump(meta, f, indent=2)

print(f"Saved to: {export_dir.resolve()}")
